<a href="https://colab.research.google.com/github/jaejunchoe/HAIDS-Lab/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import numpy as np
import pandas as pd
import torch
from nltk import word_tokenize
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate


# Dataset Definition -> Role: Data load, Review preprocessing, Transforming embedding
class ReviewDataset(Dataset):

    # 데이터에 대한 리뷰 길이 10,000으로 제한하고, embed_size 기반으로 padding, unknown, delimiter(구분자) 데이터 설정하는 함수
    def __init__(self, csv_path, emb_path, max_len=10000, embed_size=64):
        """
        데이터셋 클래스
        :param csv_path: 전처리된 CSV 파일 경로
        :param emb_path: 단어 임베딩 파일 경로
        :param max_len: 리뷰 최대 길이
        :param embed_size: 임베딩 차원
        """
        self.dataset = pd.read_csv(csv_path, header=None, names=['userID', 'itemID', 'review', 'rating'])

        ## 단어 임베딩 로드
        with open(emb_path, 'rb') as f:
            self.word_emb = pickle.load(f)

        # 임베딩 벡터 설정
        self.pad = np.zeros(embed_size)
        self.unknown = np.random.uniform(0, 1, embed_size)
        self.delimiter = np.random.uniform(0, 1, embed_size)

        # 하이퍼파라미터
        self.max_len = max_len
        self.embed_size = embed_size


    # user와 item review에 대해 preprocessing + preprocessing된 review와 rating을 반환하는 함수
    def __getitem__(self, index):
        """
        하나의 데이터 샘플을 가져옵니다.
        """
        row = self.dataset.loc[index]
        user_id = row['userID']
        item_id = row['itemID']

        # 리뷰 데이터
        user_review = self.preprocess_review(user_id, "User")
        item_review = self.preprocess_review(item_id, "Item")

        # 평점
        rating = torch.tensor(row['rating'], dtype=torch.float)

        return torch.tensor(user_review, dtype=torch.float), \
               torch.tensor(item_review, dtype=torch.float), \
               rating


    # 데이터셋의 총 길이(샘플 개수)를 반환 -> 데이터셋의 크기 확인
    def __len__(self):
        return len(self.dataset)



    # user 또는 item id에 맞는 review를 읽고 토큰화한 뒤 word_emb에서 embedding vector로 변환하고 max_len에 맞추어 처리하는 함수
    def preprocess_review(self, entity_id, entity_type):
        """
        리뷰 파일을 로드하고 임베딩을 적용합니다.
        """
        file_path = f"data/{entity_type}/{entity_id}.tsv"                   # entity_type: user or item에 대한 문자열로 user와 item을 구분
                                                                            # entity_id: 해당 user의 id or item의 id
                                                                            # 해당 부분들은 동적으로 생성되는 구조 = file_path



        try:
            reviews = pd.read_csv(file_path, sep='\t', header=None)
        except Exception:
            return [self.pad] * self.max_len                                # 만약 파일이 존재하지 않다면, [self.pad] * self.max_len로 반환.

        total_review = []
        for review_str in reviews[0][:100]:  # 최대 100개의 리뷰 단어
            tokens = word_tokenize(review_str)
            for word in tokens:
                if word in self.word_emb:
                    total_review.append(self.word_emb[word])                # word_emb에 있는 단어에는 embedding vector로 변환
                else:
                    total_review.append(self.unknown)                       # word_emb에서 없는 단어의 경우 unknown vector로 사용
            total_review.append(self.delimiter)                             # 각 review 문장 끝날 때마다 구분 벡터로 리뷰 간의 경계선 역할을 진행




        ## 리뷰 길이 조정 (패딩 또는 자르기) -> max_len(10,000)보다 길면 자르고 짧으면 pad로 padding
        if len(total_review) < self.max_len:
            pad_len = self.max_len - len(total_review)
            total_review += [self.pad] * pad_len
        else:
            total_review = total_review[:self.max_len]

        return np.array(total_review)                                       # numpy 배열로 변환하여 반환하고 이 결과는 user or item의 review를 vector화한 max_len(10,000)의 길이 배열


# Batch 생성 시 None 데이터를 제거 -> DataLoader에서 batch를 만들 때, None 값이 포함된 데이터는 제거하는 함수
def my_collate(batch):
    """
    None 데이터를 필터링하는 Collate 함수.
    """
    batch = list(filter(lambda x: x is not None, batch))                    # batch = DataLoader로부터 전달된 데이터 리스트
    return default_collate(batch)                                           # default_collate: pytorch에서 제공하는 배치 생성 함수로, tensor 형태로 변환



# DataLoader를 생성하는 Pytorch 함수 -> ReviewDataset 클래스를 기반으로 batch 단위의 DataLoader를 반환하는 역할
def get_loader(csv_path, emb_path, batch_size=32, shuffle=True, num_workers=2):
    """
    데이터 로더 생성 함수.
    :param csv_path: 전처리된 CSV 파일 경로
    :param emb_path: 단어 임베딩 파일 경로
    """
    dataset = ReviewDataset(csv_path, emb_path)                             # ReviewDataset 객체를 생성해 데이터셋을 초기화함.
    data_loader = DataLoader(dataset=dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers,
                             collate_fn=my_collate)                         # collate_fn: 배치 생성 시 사용할 함수이며, def my_collate를 사용한다.
    return data_loader
